### Daily extremes via GEV fitting ###

Fitting GEV's to the time series of block minima/maxima from the model and observations.

Start with temperature over the UK, as we are happy with how this is processed.

Next look at 10m wind speed for the GB box.

In [1]:
%matplotlib inline

# Local imports
import os
import sys
import time
import argparse
import warnings

# Third-party imports
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import shapely.geometry
import cartopy.io.shapereader as shpreader
import iris

# Specific imports
from tqdm import tqdm
from datetime import datetime, timedelta

from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error, r2_score

# Suppress warnings
warnings.filterwarnings('ignore')

/home/users/benhutch/.conda/envs/bens-conda-env2/lib/python3.11/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
output_dir_dfs = "/gws/nopw/j04/canari/users/benhutch/unseen/saved_dfs"

# import the temperature data
df_model_tas = pd.read_csv(
    f"{output_dir_dfs}/HadGEM3-GC31-MM_dcppA-hindcast_tas_United_Kingdom_1960-2018_day.csv"
)

# set up the djf winter years
djf_winter_years = np.arange(1, 11 + 1)

# set up a new dataframe to append to
df_model_tas_djf = pd.DataFrame()

# loop over the winter years
for i, wyear in enumerate(djf_winter_years):
    # set up the leads to extract
    leads = np.arange(31 + (i * 360), 31 + 90 + (i * 360))

    # extract the data
    df_model_leads_this = df_model_tas[df_model_tas["lead"].isin(leads)]

    # include a new column
    df_model_leads_this["winter_year"] = wyear

    # Append to the new df
    df_model_tas_djf = pd.concat([df_model_tas_djf, df_model_leads_this])

In [3]:
# subset model djf tas to the first winter
df_model_tas_djf_first = df_model_tas_djf[df_model_tas_djf["winter_year"] == 1]

# add the column for effective dec year to the model data
df_model_tas_djf_first["effective_dec_year"] = df_model_tas_djf_first["init_year"] + (df_model_tas_djf_first["winter_year"] - 1)

In [4]:
# import the observed data
# load the obs data
df_obs_tas = pd.read_csv(
    f"{output_dir_dfs}/ERA5_tas_United_Kingdom_1960-2018_daily_2024-11-26.csv"
)

# Convert the 'time' column to datetime, assuming it represents days since "1950-01-01 00:00:00"
df_obs_tas["time"] = pd.to_datetime(df_obs_tas["time"], origin="1950-01-01", unit="D")

# subset the obs data to D, J, F
df_obs_tas = df_obs_tas[df_obs_tas["time"].dt.month.isin([12, 1, 2])]

# new column for temp in C
df_obs_tas["data_c"] = df_obs_tas["data"] - 273.15

In [5]:
def determine_effective_dec_year(row):
    year = row["time"].year
    month = row["time"].month
    if month in [1, 2, 3]:
        return year - 1
    elif month in [10, 11, 12]:
        return year
    else:
        return None
    
# apply the effective dec year to the obs data
df_obs_tas["effective_dec_year"] = df_obs_tas.apply(
    lambda row: determine_effective_dec_year(row), axis=1
)

In [6]:
# Limit both datasets between 1960 and 2019
df_model_tas_djf_first = df_model_tas_djf_first[
    df_model_tas_djf_first["effective_dec_year"].between(1960, 2017)
]
df_obs_tas = df_obs_tas[df_obs_tas["effective_dec_year"].between(1960, 2017)]

# create new columns for data_c inh df_model_tas_djf_first
df_model_tas_djf_first["data_c"] = df_model_tas_djf_first["data"] - 273.15

In [7]:
df_obs_tas.tail()

,time,data,data_c,effective_dec_year
21239,2018-02-24,275.004977,1.854977,2017
21240,2018-02-25,274.473160,1.323160,2017
21241,2018-02-26,273.135263,-0.014737,2017
21242,2018-02-27,271.899134,-1.250866,2017
21243,2018-02-28,269.332502,-3.817498,2017


In [10]:
df_model_tas_djf_first.tail()

,init_year,member,lead,data,winter_year,effective_dec_year,data_c
2171375,2017,10,116,273.345904,1,2017,0.195904
2171376,2017,10,117,273.571250,1,2017,0.421250
2171377,2017,10,118,274.187873,1,2017,1.037873
2171378,2017,10,119,275.065096,1,2017,1.915096
2171379,2017,10,120,276.088821,1,2017,2.938821


In [12]:
# Now calculate the lowest temperature day for each effective dec year
df_obs_lowest_tas_day_djf = pd.DataFrame()

# loop over the effective dec years
for year in df_obs_tas["effective_dec_year"].unique():
    # subset the data
    df_this = df_obs_tas[
        df_obs_tas["effective_dec_year"] == year
    ]

    # find the lowest wind day
    lowest_tas_day = df_this["data_c"].idxmin()

    # Create a new dataframe
    df_new = pd.DataFrame(
        {
            "effective_dec_year": [year],
            "time": [lowest_tas_day],
            "data": [df_this.loc[lowest_tas_day, "data_c"]],
        }
    )

    # Append to the new dataframe
    df_obs_lowest_tas_day_djf = pd.concat([df_obs_lowest_tas_day_djf, df_new])

In [11]:
# same for the model data
df_model_lowest_tas_day_djf = pd.DataFrame()

# loop over the effective dec years
for year in df_model_tas_djf_first["effective_dec_year"].unique():
    for member in df_model_tas_djf_first["member"].unique():
        # subset the data
        df_this = df_model_tas_djf_first[
            (df_model_tas_djf_first["effective_dec_year"] == year)
            & (df_model_tas_djf_first["member"] == member)
        ]

        # find the lowest wind day
        lowest_tas_day = df_this["data_c"].idxmin()

        # Create a new dataframe
        df_new = pd.DataFrame(
            {
                "effective_dec_year": [year],
                "member": [member],
                "time": [lowest_tas_day],
                "data": [df_this.loc[lowest_tas_day, "data_c"]],
            }
        )

        # Append to the new dataframe
        df_model_lowest_tas_day_djf = pd.concat([df_model_lowest_tas_day_djf, df_new])